# JLCPCB Local Database Search

Requires parts database to already be downloaded and unzipped ```jlcpcb-components.sqlite3```

[Setup Python Virtual Environment in VSCode](/venv-setup.md)

In [ ]:
%pip install requests

In [3]:
import sqlite3
import json
import requests
import os


def download_file(url, filename):
    try:
        # Check if the file already exists
        if os.path.exists(filename):
            # Delete the existing file
            os.remove(filename)
            print(f"Deleted existing file: {filename}")

        response = requests.get(f"{url}/{filename}", stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes
        with open(filename, "wb") as f:
            for chunk in response.iter_content(None):
                f.write(chunk)
        print(f"Downloaded {url} to {filename}")
    except requests.RequestException as e:
        print(f"Download {url} failed: {e}")


# URL of the SQLite database file
url = "https://cdfer.github.io/jlcpcb-parts-database"

# Local file path to save the database file
local_file = "jlcpcb-components.sqlite3"

# Download the file
download_file(url, local_file)

Deleted existing file: jlcpcb-components.sqlite3
Downloaded https://cdfer.github.io/jlcpcb-parts-database to jlcpcb-components.sqlite3


In [4]:
conn = sqlite3.connect(r"jlcpcb-components.sqlite3")
cur = conn.cursor()

cur.execute(
    "PRAGMA temp_store = MEMORY"
)  # Store temporary tables and indices in memory for faster access
cur.execute("PRAGMA mmap_size = 2147483648")  # Set the maximum memory map size to 2 GiB

In [6]:
# Search terms
search_terms = ["2919662"]

query = """
    SELECT c.price, c.stock, f.*
    FROM components c
    JOIN components_fts f ON c.lcsc = f.lcsc
    WHERE f.mfr LIKE ? OR f.description LIKE ? OR f.datasheet LIKE ? OR f.lcsc LIKE ?
"""

rows = []
for term in search_terms:
    # Execute the query
    cur.execute(
        query,
        (f"%{term}%",) * 4,
    )

    # Fetch the results and append to rows
    rows.extend(cur.fetchall())

# Sort the rows based on the price
rows.sort(
    key=lambda row: json.loads(row[0])[0]["price"],
    reverse=True,
)

# Print the results
for i, row in enumerate(rows):
    price_data = json.loads(row[0])
    print(f"{row[3]}")
    print(f"Price: {round(price_data[0]['price'], 2)}USD")
    print(f"Stock: {row[1]}")
    print(f"Package: {row[4]}")
    print(f"Description: {row[5]}")
    print(f"https://jlcpcb.com/partdetail/C{row[2]}")
    print()

KH-TYPE-C-L13.7-6P
Price: 0.23USD
Stock: 49
Package: Plugin
Description: Direct Insert 6P Type-C Plugin  USB Connectors ROHS
https://jlcpcb.com/partdetail/C2919662



In [17]:
# Close the database connection
conn.close()